In [ ]:
!pip install gymnasium
!pip install numpy
!pip install pygame
!pip install random
!pip install tqdm
!pip install matplotlib

In [1]:
import gymnasium as gym
import numpy as np
import pygame
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import time

In [2]:
class MazeGameEnv(gym.Env):
    def __init__(self, maze, alpha=0.1, gamma=0.99, epsilon=0.1):
        super(MazeGameEnv, self).__init__()
        self.maze = np.array(maze)
        self.start_pos = tuple(np.argwhere(self.maze == 'S')[0])
        self.goal_pos = tuple(np.argwhere(self.maze == 'G')[0])
        self.current_pos = self.start_pos
        self.num_rows, self.num_cols = self.maze.shape

        self.alpha = alpha  # Taxa de aprendizado
        self.gamma = gamma  # Fator de desconto
        self.epsilon = epsilon  # Probabilidade de exploração

        # Define ação como Discrete com 4 ações (cima, baixo, esquerda, direita)
        self.action_space = gym.spaces.Discrete(4)

        # Defina o espaço de observação como uma tupla com o número de linhas e colunas
        self.observation_space = gym.spaces.Tuple((gym.spaces.Discrete(self.num_rows), gym.spaces.Discrete(self.num_cols)))

        # Inicialize o ambiente Pygame
        pygame.init()

        # Defina cores
        self.WHITE = (255, 255, 255)
        self.GREEN = (0, 255, 0)
        self.RED = (255, 0, 0)
        self.BLACK = (0, 0, 0)
        self.PURPLE = (255, 0, 255)
        self.BLUE = (0, 0, 255)

    def reset(self, num_obstacles=None):
        self.reset_obstacles(num_obstacles)
        row, col = np.random.randint(0, self.num_rows), np.random.randint(0, self.num_cols)
        while self.maze[row, col] in ['#', 'G']:  # Garante que a posição inicial não é um obstáculo ou a posição final
            row, col = np.random.randint(0, self.num_rows), np.random.randint(0, self.num_cols)
        self.maze[self.start_pos] = '.'  # Limpa a posição inicial antiga
        self.start_pos = (row, col)
        self.maze[row, col] = 'S'  # Define a nova posição inicial
        self.current_pos = self.start_pos
        return self.current_pos
    
    def reset_obstacles(self, num_obstacles = 0):
        for row in range(self.num_rows):
            for col in range(self.num_cols):
                if self.maze[row, col] == 'R':
                    self.maze[row, col] = '.'
        
        for _ in range(num_obstacles):
            row, col = np.random.randint(0, self.num_rows), np.random.randint(0, self.num_cols)
            while self.maze[row, col] in ['S', 'G', '#','R']:
                row, col = np.random.randint(0, self.num_rows), np.random.randint(0, self.num_cols)
            self.maze[row, col] = 'R'

    def step(self, action, reward = 1):
        if action == 0:  # Cima
            self.move('up')
        elif action == 1:  # Baixo
            self.move('down')
        elif action == 2:  # Esquerda
            self.move('left')
        elif action == 3:  # Direita
            self.move('right')

        obs = self.current_pos
        #
        # reward = 1 if self.current_pos == self.goal_pos else 0
        # distance_to_goal = abs(self.current_pos[0] - self.goal_pos[0]) + abs(self.current_pos[1] - self.goal_pos[1])
        # reward = 1 / (distance_to_goal + 1)
        # done = self.current_pos == self.goal_pos
        #
        if self.current_pos == self.goal_pos:
            reward = 10
            done = True
        else:
            distance_to_goal = abs(self.current_pos[0] - self.goal_pos[0]) + abs(self.current_pos[1] - self.goal_pos[1])
            reward = -distance_to_goal
            done = False
            
        info = {}

        return obs, reward, done, info
    
    def move(self, action):
        new_pos = list(self.current_pos)

        if action == 'up':
            new_pos[0] -= 1
        elif action == 'down':
            new_pos[0] += 1
        elif action == 'left':
            new_pos[1] -= 1
        elif action == 'right':
            new_pos[1] += 1

        new_pos = tuple(new_pos)

        if self.is_valid_position(new_pos[0], new_pos[1]):
            self.current_pos = new_pos

    def is_valid_position(self, row, col):
        return 0 <= row < self.num_rows and 0 <= col < self.num_cols and self.maze[row, col] != '#' and self.maze[row, col] != 'R'


    def render(self, mode='human', path=None):
        if mode == 'human':
            self.draw_maze(path)
            pygame.display.update()
            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
        
        if mode == 'blank':
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()

    def play_path(self, path, delay=1):
        for position in path:
            self.current_pos = position
            self.render(mode='human', path=path)
            time.sleep(delay)

    def draw_maze(self, path = None):
        self.cell_size = 50
        self.screen_width = self.num_cols * self.cell_size
        self.screen_height = self.num_rows * self.cell_size
        self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))
        self.screen.fill(self.WHITE)
        
        for row in range(self.num_rows):
            for col in range(self.num_cols):
                cell_left = col * self.cell_size
                cell_top = row * self.cell_size

                if self.maze[row, col] == '#' or self.maze[row, col] == 'R':
                    pygame.draw.rect(self.screen, self.BLACK, (cell_left, cell_top, self.cell_size, self.cell_size))
                elif self.maze[row, col] == 'S':
                    pygame.draw.rect(self.screen, self.GREEN, (cell_left, cell_top, self.cell_size, self.cell_size))
                elif self.maze[row, col] == 'G':
                    pygame.draw.rect(self.screen, self.RED, (cell_left, cell_top, self.cell_size, self.cell_size))
                    
        if path:
            for i in range(len(path)-1):
                start = path[i]
                end = path[i+1]
                self.draw_arrow(start, end)

        current_row, current_col = self.current_pos
        pygame.draw.rect(self.screen, self.PURPLE, (current_col * self.cell_size, current_row * self.cell_size, self.cell_size, self.cell_size))

    def draw_arrow(self, start, end):
        start_x = start[1] * self.cell_size + self.cell_size // 2
        start_y = start[0] * self.cell_size + self.cell_size // 2
        end_x = end[1] * self.cell_size + self.cell_size // 2
        end_y = end[0] * self.cell_size + self.cell_size // 2

        pygame.draw.line(self.screen, self.BLUE, (start_x, start_y), (end_x, end_y), 2)

    def close(self):
        pygame.quit()


In [5]:
maze = [
    ['S', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '#', '.', '#', '.', '.', '.'],
    ['.', '.', '.', '.', '#', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '#', '.', '.', '.', '#', '.'],
    ['.', '.', '.', '.', '#', '.', '.', '.', '.', '.'],
    ['.', '#', '#', '#', '#', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '.', '#', '.', '.', '.', '.'],
    ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
    ['.', '#', '.', '.', '.', '.', '.', '.', '.', 'G'],
]

In [6]:
maze = [
    ['S','.','.','.','.','.'],
    ['.','.','.','#','.','.'],
    ['.','.','.','#','.','.'],
    ['.','.','.','#','.','.'],
    ['.','#','#','#','.','.'],
    ['.','.','.','.','.','G']
]

In [3]:
maze = [
    ['S','.','#','.','#','.'],
    ['.','.','.','.','#','.'],
    ['.','#','#','.','#','.'],
    ['.','#','#','.','#','.'],
    ['.','.','.','.','#','.'],
    ['#','.','.','.','.','G']
]

In [3]:
maze = [
    ['S','.','.','.','.','.'],
    ['.','.','.','.','.','.'],
    ['.','.','#','#','.','.'],
    ['.','.','#','#','.','.'],
    ['.','.','.','.','.','.'],
    ['.','.','.','.','.','G']
]

In [ ]:
env = MazeGameEnv(maze)

done = False
state = env.reset()
env.render()

while not done:
    action = env.action_space.sample()  # Exemplo: selecione uma ação aleatória
    state, reward, done, _ = env.step(action)
    print(state, reward, done)
    env.render()
    pygame.time.delay(100)

print("Done!")
env.close()

In [4]:
class MonteCarlo:
    def __init__(self, env, learning_rate=0.1, discount_factor=0.99, exploration_prob=0.1):
        self.env = env
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_prob = exploration_prob
        
        self.Q_table = {}
        for row in range(env.num_rows):
            for col in range(env.num_cols):
                state = (row, col)    #destino
                self.Q_table[state] = np.zeros(env.action_space.n)
    
        self.mc_acoe = [0,1,2,3]
        self.mc_reward = -1
        self.VI = np.zeros((env.num_cols, env.num_rows))
        self.otimo = []
        self.converg = 80
        
        self.dest = []
        self.dest.append(list(state))
        self.gamm_arr = {(x, y):list() for x in range(env.num_cols) for y in range(env.num_rows)}
        self.vabs = {(x, y):list() for x in range(env.num_cols) for y in range(env.num_rows)}
        self.sta = [[x, y] for x in range(env.num_cols) for y in range(env.num_rows)] 
        
    def epocas(self):
        begin_state = env.start_pos  
        epocas = []
        while True:
            # Se estado igual destino encerra a epoca
            if list(begin_state) in self.dest:
                self.env.reset()
                return epocas
            # Seleciona de forma randomica uma ação (cima , esquerda, ...   )
            action = random.choice(self.mc_acoe)
            
            # Retorna os novos estados que podem ser utilizados
            new_state, _, _, _ = self.env.step(action, 0)  # valida se tem parede ou borda

            # Cria lista das epocas
            epocas.append([list(begin_state), action, self.mc_reward, list(new_state)])

            begin_state = new_state
        
    def MC(self, mc_interactions):
        conv_epoc = []
        for z in tqdm(range(mc_interactions)):
            epoc = self.epocas()
            ga = 0
            vi_ant = None
            count = 0
            for i, passo in enumerate(epoc[::-1]):
                ga = env.gamma*ga + passo[2]
                # Começa pelas epocas proximas ao destino
                if passo[0] not in [x[0] for x in epoc[::-1][len(epoc)-i:]]:
                    # Pega posicao que vai ser calculada
                    pos = (passo[0][0], passo[0][1])
                    # Monta uma lista de gamma para posição  
                    self.gamm_arr[pos].append(ga)
                    # Calcula a media de todos os gammas da posicao
                    newValue = np.average(self.gamm_arr[pos])
                    # Monta uma lista na posição com o calculo do valor absoluto da diferença entre as posições - antiga e nova
                    self.vabs[pos[0], pos[1]].append(np.abs(self.VI[pos[0], pos[1]]-newValue))
                    # lista com os valores de interacao 
                    self.VI[pos[0], pos[1]] = newValue
                    
            if z >= self.converg:  # convergencia de acordo com o grafico
                conv_epoc.append(epoc)
            
            if z in [100,200,300, 400, mc_interactions-1]:
                print("Iteration {}".format(z+1))
                print(self.VI)
                print("")
                        
            vabs_ant = self.vabs
        
        plt.figure(figsize=(20,10))
        all_series = [list(x)[:100] for x in self.vabs.values()]
        for series in all_series:
            plt.plot(series)
            
        list(map(len, conv_epoc))        
    
        def get_line_len(conv_epoc):
            return len(conv_epoc),conv_epoc
        
        list_converg = sorted(list(map(get_line_len, conv_epoc)))

        print (f'O menos número de passos atingidos em {mc_interactions} interações foram : {list_converg[0][0]}')
        
        print (f'Ocorreu na epoca  {(self.converg + list_converg[0][0])} e é considerado como o resultado ótimo para este treinamento')
        
        steps = list_converg[0][1]
        
        bob_volta_casa = []
        
        for item in steps:
            bob_volta_casa.append((item[3][0], item[3][1]))
        
        return bob_volta_casa
            
    def find_optimal_path(self):
        state = env.start_pos
        optimal_path = [state]

        while state != env.goal_pos:
            action = np.argmax(self.Q_table[state])
            env.move(action)
            state = env.current_pos
            optimal_path.append(state)

        print(self.VI)
        return optimal_path
                

In [ ]:
env = MazeGameEnv(maze, alpha=0.1, gamma=0.6, epsilon=0.1)
conf = MonteCarlo(env)
optimal_path = conf.MC(500)

# # Encontre o caminho ótimo do estado inicial ao estado de destino
print("Optimal Path:", optimal_path)

In [4]:
class QLearningAgent:
    def __init__(self, env, alpha=0.1, gamma=0.99, epsilon=0.1):
        self.env = env
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        
        self.Q = {}
        for row in range(env.num_rows):
            for col in range(env.num_cols):
                self.Q[(row, col)] = {}
                for a in range(env.action_space.n):
                    self.Q[(row, col)][a] = 0
    
    def choose_action(self, state):
        if np.random.uniform(0, 1) < self.epsilon:
            return self.env.action_space.sample()  # Explore
        else:
            return max(self.Q[state], key=self.Q[state].get)  # Exploit
    
    def learn(self, state, action, reward, next_state):
        predict = self.Q[state][action]
        target = reward
        if next_state:
            target = reward + self.gamma * max(self.Q[next_state].values())
        
        self.Q[state][action] += self.alpha * (target - predict)
    
    def train(self, episodes, num_obstacles=0, max_steps=10000):
        for episode in range(episodes):
            path = []  # Lista para armazenar o caminho do episódio atual
            state = self.env.reset(num_obstacles)
            path.append(state)
            done = False
            step_count = 0  # Contador de passos
            while not done:
                if step_count >= max_steps:  # Verifique o limite de passos
                    break
                action = self.choose_action(state)
                next_state, reward, done, _ = self.env.step(action)
                path.append(next_state)  # Adicione o estado ao caminho
                self.learn(state, action, reward, None if done else next_state)
                state = next_state
                self.env.render()
                step_count += 1  # Incremente o contador de passos
            self.last_path = path  # Armazene o caminho do último episódio
        self.env.close()

            
    def get_last_path(self):
        return self.last_path


In [7]:
env = MazeGameEnv(maze)
agent = QLearningAgent(env)
agent.train(1000)

last_path = agent.get_last_path()
env.play_path(last_path)
env.close()

In [ ]:
env = MazeGameEnv(maze)
agent = QLearningAgent(env)
agent.train(1000, 2)

last_path = agent.get_last_path()
env.play_path(last_path)
env.close()